In [ ]:
pip install fairseq

In [3]:
cd examples/translation/

/home/eduardo/PycharmProjects/fairseq-dynamic-routing-plugin/examples/translation


In [5]:
pip install bitarray

  Using cached bitarray-2.3.4.tar.gz (88 kB)
  ERROR: Command errored out with exit status 1:
   command: /home/eduardo/PycharmProjects/fairseq-dynamic-routing-plugin/venv/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-t6z8qw3v/bitarray_f77975f781954832bc9341aa32d65d5a/setup.py'"'"'; __file__='"'"'/tmp/pip-install-t6z8qw3v/bitarray_f77975f781954832bc9341aa32d65d5a/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-y12zlnzn
       cwd: /tmp/pip-install-t6z8qw3v/bitarray_f77975f781954832bc9341aa32d65d5a/
  Complete output (26 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.7
  creating build/lib.linux-x86_64-3.7/bitarray
  

In [6]:

! bash prepare-iwslt14.sh
! cd ../..

# Preprocess/binarize the data
! TEXT=examples/translation/iwslt14.tokenized.de-en
! fairseq-preprocess --source-lang de --target-lang en \
    --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test \
    --destdir data-bin/iwslt14.tokenized.de-en \
    --workers 20

Cloning Moses github repository (for tokenization scripts)...
fatal: destination path 'mosesdecoder' already exists and is not an empty directory.
Cloning Subword NMT repository (for BPE pre-processing)...
fatal: destination path 'subword-nmt' already exists and is not an empty directory.
--2021-10-14 01:55:56--  http://dl.fbaipublicfiles.com/fairseq/data/iwslt14/de-en.tgz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19982877 (19M) [application/x-tar]
Saving to: ‘de-en.tgz.1’

de-en.tgz.1         100%[===================>]  19.06M  4.64MB/s    in 4.7s    

2021-10-14 01:56:01 (4.06 MB/s) - ‘de-en.tgz.1’ saved [19982877/19982877]

Data successfully downloaded.
de-en/
de-en/IWSLT14.TED.dev2010.de-en.de.xml
de-en/IWSLT14.TED.dev2010.de-en.en.xml
de-en/IWSLT14.TED.tst2010.de-en.de.x

In [ ]:
! fairseq-train data-bin/iwslt14.tokenized.de-en \
    --user-dir examples/dynamic_routing/dynamic_routing_src \
    --arch capsnet_transformer --share-all-embeddings \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric